In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import pandas as pd
import re
import os
import pickle
import sqlite3
import numpy as np
import datetime
#import html5lib
headers = {'User-Agent':'Mozilla/5.0'}
import time
from nba.nba import data

#need 18 dec - april  
#need all of 20  
21 past feb 17  

In [3]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()

# Select the dictionary for the Pacers, which contains their team ID
bos = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
bos_id = bos['id']
print(f'boston_id: {bos_id}')

boston_id: 1610612738


In [4]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType


gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable = '00',
                                               date_from_nullable = '10/01/2017',
                                               season_type_nullable = 'Regular Season'
                                                
                                               )  
game_dates = gamefinder.get_data_frames()[0]['GAME_DATE'].unique()
games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game_ids = list(set([v for l in games for k,v in l.items() if k=='GAME_ID']))
# game = games[0]
# game_id = game['GAME_ID']
# game_matchup = game['MATCHUP']

# print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')

### Shooting information

In [5]:
from nba_api.stats.endpoints import CommonTeamRoster,LeagueDashPlayerShotLocations,CommonPlayerInfo,PlayerProfileV2,PlayerNextNGames,DraftCombineStats,LeagueDashPtStats,LeagueDashTeamShotLocations
#shots = LeagueDashPlayerShotLocations(date_from_nullable = '02/13/2024',team_id_nullable = 1610612738).get_data_frames()

In [ ]:
DraftCombinePlayerAnthro().get_data_frames()[0]

In [28]:
octdf = pd.DataFrame([pd.to_datetime(x) for x in game_dates  if pd.to_datetime(x).month==10],columns = ['day'])

In [36]:
octdf['year'] = [x.year for x in octdf.day]

In [56]:
starts = [pd.to_datetime(x) for x in octdf.groupby('year').day.min().values]

In [57]:
for date in game_dates:
    if date in starts:
        print(True)

In [54]:
np.isin(pd.to_datetime(date),starts)

array(False)

In [51]:
np.any(starts == date)

/var/folders/y5/cm5rrvr94yx214kzmtmsbvz40000gn/T/ipykernel_16922/826002457.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  np.any(starts == date)


False

In [59]:
date

'2023-03-01'

In [60]:
missing = []
l = []
final = pd.DataFrame()
t = time.time()
games = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
print('starting at {}'.format(time.strftime('%H:%M')))
for ct,date in enumerate(game_dates):
    try:
        d = pd.to_datetime(date)
        season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])

        rbs = LeagueDashPtStats(pt_measure_type='Rebounding',player_or_team='Player',
                               date_from_nullable=date,
                               date_to_nullable = date,
                               season = season
                               ).get_data_frames()[0][['PLAYER_ID','TEAM_ID','OREB','OREB_CONTEST','OREB_CHANCES','OREB_CHANCE_DEFER','AVG_OREB_DIST',
                             'DREB','DREB_CONTEST','DREB_CHANCES','DREB_CHANCE_DEFER','AVG_DREB_DIST']]
        temp = rbs.merge(games[games.GAME_DATE==d],how='left',on=['TEAM_ID'])
        final = pd.concat([final,temp])
    except:
        missing.append(date)
    if ct % 18 == 0:
        time.sleep(np.random.choice(range(1,12),1)[0])
    elif (ct % 25 == 0) & (ct!=0):
        print('through {} days in {:.2f} with {:,} missing'.format(ct,((time.time() -t)/60),len(missing)))
        
    l.append(final.to_dict(orient='records'))
    if d in starts:
        pickle.dump(l,open('nba/data/pickle/plyrRebs{}.pkl'.format(season),'wb'))   
        print('\tsaved {} season rebs'.format(season))
        l = []
        final = pd.DataFrame()

print('finished at {}'.format(time.strftime('%H:%M')))
print('missing {:_}'.format(len(missing)))

starting at 20:23
through 25 days in 0.24 with 0 missing
through 50 days in 0.37 with 0 missing
through 75 days in 0.53 with 0 missing
through 100 days in 0.66 with 0 missing
through 125 days in 0.84 with 0 missing
through 150 days in 1.13 with 0 missing
saved 2023-24 season rebs
through 175 days in 1.58 with 0 missing
through 200 days in 1.95 with 0 missing
through 225 days in 3.18 with 0 missing
through 250 days in 3.60 with 0 missing
through 275 days in 4.26 with 0 missing
through 300 days in 4.75 with 0 missing
saved 2022-23 season rebs
through 325 days in 8.45 with 0 missing
through 350 days in 8.84 with 0 missing
through 375 days in 9.29 with 0 missing
through 400 days in 9.74 with 0 missing
through 425 days in 10.22 with 0 missing
through 475 days in 11.13 with 0 missing
saved 2021-22 season rebs
through 500 days in 11.64 with 0 missing
through 525 days in 12.21 with 0 missing
through 550 days in 12.68 with 0 missing
through 575 days in 13.15 with 0 missing
through 600 days in 1

In [62]:
os.listdir('nba/data/pickle')

['advbox.pkl',
 'plyrRebsupt800.pkl',
 'plyrRebs2019-20pkl',
 'shooting2.pkl',
 'quarter2Info.pkl',
 'shooting.pkl',
 'plyrRebs2017-18pkl',
 'plyrRebs2023-24pkl',
 'plyrRebsup701up.pkl',
 'dribbleThru500days.pkl',
 'plyrRebs2021-22pkl',
 'drbShots.pkl',
 'plyrRebs2022-23pkl',
 'baskets.pkl',
 'plyrShots.pkl',
 'quarterInfo.pkl',
 'box.pkl',
 'plyrRebs2018-19pkl']

In [64]:
rbs23 = pd.read_pickle('nba/data/pickle/plyrRebs2023-24pkl')

In [ ]:
PlyrCombine

In [ ]:
for date in game_dates:

    rbs = LeagueDashPtStats(pt_measure_type='Rebounding',player_or_team='Player',
                           date_from_nullable=date,
                           ate_to_nullable = date).get_data_frames()[0]
    rbs['GAME_DATE'] = date

In [ ]:
rbs.get_data_frames()[0][

### First Baskets

In [69]:
from nba_api.stats.endpoints import BoxScoreSummaryV2,BoxScoreTraditionalV2,BoxScoreAdvancedV2,PlayByPlayV2,LeagueDashOppPtShot

In [ ]:
box = BoxScoreAdvancedV2('0022300061')

In [ ]:
box.get_data_frames()[0]

In [7]:
rbs1 = pd.read_pickle('nba/data/pickle/plyrRebsup701up.pkl')

KeyboardInterrupt: 

In [83]:
df = LeagueDashPlayerShotLocations(date_from_nullable = date,
                             date_to_nullable = date,
                             season=season).get_data_frames()[0]
df = clean_shot_columns(df)
df['GAME_DATE'] = date
final = df.merge(temp,how='left',on=['GAME_DATE','TEAM_ID'])
l.append(final.to_dict(orient='records'))

#### player Shooting

In [85]:
missing = []
l = []
t = time.time()
print('starting at {}'.format(time.strftime('%H:%M')))
temp = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
temp = temp[temp.GAME_DATE==date]
for ct,date in enumerate(game_dates):
    d = pd.to_datetime(date)
    season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
    try:
        df = LeagueDashPlayerShotLocations(date_from_nullable = date,
                                     date_to_nullable = date,
                                     season=season).get_data_frames()[0]
        df = clean_shot_columns(df)
        df['GAME_DATE'] = date
        final = df.merge(temp,how='left',on=['GAME_DATE','TEAM_ID'])
        l.append(final.to_dict(orient='records'))
    except:
        missing.append(date)
    if ct % 18 == 0:
        time.sleep(np.random.choice(range(1,12),1)[0])
    elif (ct % 25 == 0) & (ct!=0):
        print('through {} days in {:.2f}'.format(ct,((time.time() -t)/60)))

    if d in starts:
        pickle.dump(l,open('nba/data/pickle/plyrShots{}.pkl'.format(season),'wb'))   
        print('\tsaved {} season player shots'.format(season))
        l = []
        final = pd.DataFrame()   
print('finished at {}'.format(time.strftime('%H:%M')))
print('missing {:_}'.format(len(missing)))

starting at 07:16
through 25 days in 0.30
through 50 days in 1.90
through 75 days in 3.59
through 100 days in 5.29
through 125 days in 6.89
through 150 days in 8.68
	saved 2023-24 season player shots
through 175 days in 11.26
through 200 days in 32.01
through 225 days in 68.67
through 250 days in 90.46
through 275 days in 100.97
through 300 days in 113.03
	saved 2022-23 season player shots
through 325 days in 151.75
through 350 days in 164.79
through 375 days in 183.63
through 400 days in 200.98
through 425 days in 248.86
through 475 days in 290.77
	saved 2021-22 season player shots
through 500 days in 318.51
through 525 days in 336.88
through 550 days in 338.51
through 575 days in 340.28
through 600 days in 356.80
through 625 days in 358.60
through 650 days in 360.30
through 675 days in 361.97
through 700 days in 363.65
through 725 days in 365.36
through 750 days in 367.04
through 775 days in 368.67
	saved 2019-20 season player shots
through 800 days in 370.44
through 825 days in 372.

### Shots by dribbles

In [68]:
df = LeagueDashOppPtShot(date_from_nullable = date,
                                     date_to_nullable = date,
                                     season=season).get_data_frames()[0]

NameError: name 'LeagueDashOppPtShot' is not defined

In [79]:
game_dates[:25]

array(['2024-04-14', '2024-04-12', '2024-04-11', '2024-04-10',
       '2024-04-09', '2024-04-07', '2024-04-06', '2024-04-05',
       '2024-04-04', '2024-04-03', '2024-04-02', '2024-04-01',
       '2024-03-31', '2024-03-30', '2024-03-29', '2024-03-28',
       '2024-03-27', '2024-03-26', '2024-03-25', '2024-03-24',
       '2024-03-23', '2024-03-22', '2024-03-21', '2024-03-20',
       '2024-03-19'], dtype=object)

In [75]:
l = []
drib = ['0 Dribbles','1 Dribble','2 Dribbles','3-6 Dribbles','7+ Dribbles']
missing = []
final = pd.DataFrame()
t = time.time()
games = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
print('starting at {}'.format(time.strftime('%H:%M')))

for ct,date in enumerate(game_dates):
    d = pd.to_datetime(date)
    season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
    drb = pd.DataFrame()
    try:
        for dribbleCount in drib:
            drbShots = LeagueDashOppPtShot(date_from_nullable = date,
                                date_to_nullable = date,
                                           season=season,
                                dribble_range_nullable=dribbleCount).get_data_frames()[0]
            df = lgDashOppColums(drbShots,dribbleCount)
            drb = pd.concat([drb,df])
            time.sleep(1)
        drb = drb.merge(games[games.GAME_DATE==d],how='left',on=['TEAM_ID'])
        drb = drb.groupby(['TEAM_ID','GAME_ID','GAME_DATE']).sum().reset_index()
    except:
        missing.append(date)
    final = pd.concat([final,drb])
    if ct % 18 == 0:
        time.sleep(np.random.choice(range(1,12),1)[0])
    elif (ct % 25 == 0) & (ct!=0):
        print('through {} days in {:.2f}'.format(ct,((time.time() -t)/60)))
    l.append(drb.to_dict(orient='records'))

    if d in starts:
        pickle.dump(l,open('nba/data/pickle/drbShots{}.pkl'.format(season),'wb'))   
        print('\tsaved {} season drible shots'.format(season))
        l = []
        drb = pd.DataFrame()
print('finished at {}'.format(time.strftime('%H:%M')))
print('missing {:_}'.format(len(missing)))

starting at 20:39
through 25 days in 0.53
through 50 days in 0.93
through 75 days in 1.53
through 100 days in 1.96
through 125 days in 2.35
through 150 days in 2.89
	saved 2023-24 season drible shots
through 175 days in 3.50
through 200 days in 4.08
through 225 days in 4.59
through 250 days in 5.01
through 275 days in 5.63
through 300 days in 6.05
	saved 2022-23 season drible shots
through 325 days in 6.58
through 350 days in 6.96
through 375 days in 7.38
through 400 days in 8.06
through 425 days in 8.47
through 475 days in 9.29
	saved 2021-22 season drible shots
through 500 days in 9.67
through 525 days in 10.24
through 550 days in 10.78
through 575 days in 11.20
through 600 days in 11.78
through 625 days in 12.27
through 650 days in 12.79
through 675 days in 13.31
through 700 days in 13.84
through 725 days in 14.28
through 750 days in 14.82
through 775 days in 15.38
	saved 2019-20 season drible shots
through 800 days in 15.98
through 825 days in 16.39
through 850 days in 17.08
throug

In [74]:
#team defense shooting
l = []
missing = []
t = time.time()
print('starting at {}'.format(time.strftime('%H:%M')))

for ct,date in enumerate(game_dates):
    try:
        d = pd.to_datetime(date)
        season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
        oppShots = LeagueDashTeamShotLocations(measure_type_simple='Opponent',
                                          date_from_nullable = date,
                                           date_to_nullable = date,
                                       season=season
                           ).get_data_frames()[0]
        oppShots.columns = ['{}_{}'.format(re.sub(' |-','_',a),b) if a!='' else b for a,b in oppShots.columns]
        oppShots = oppShots.filter([col for col in oppShots.columns if re.search('_PCT$',col)==None])
        oppShots['GAME_DATE'] = date
        wide =  LeagueDashOppPtShot(date_from_nullable = date,
                          date_to_nullable = date, season=season,
                              close_def_dist_range_nullable = '6+ Feet - Wide Open').get_data_frames()[0]
        wide = wide.filter([col for col in wide.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                    (wide[col].dtype!=object)] )
        wide.columns = [col if re.search('FG',col)==None else 'WIDEOPEN_{}'.format(col) for col in wide.columns]
        wide['GAME_DATE'] = date

        op = LeagueDashOppPtShot(date_from_nullable = date,
                          date_to_nullable = date,season=season,
                              close_def_dist_range_nullable = '4-6 Feet - Open').get_data_frames()[0]
        op = op.filter([col for col in op.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                    (op[col].dtype!=object)] )
        op.columns = [col if re.search('FG',col)==None  else 'OPEN_{}'.format(col) for col in op.columns]
        op['GAME_DATE'] = date
        final = oppShots.merge(wide.merge(op,how='left',on=['TEAM_ID','GAME_DATE']),how='left',on=['TEAM_ID','GAME_DATE'])
        l.append(final.to_dict(orient='records'))
        if ct % 18 == 0:
            time.sleep(np.random.choice(range(1,12),1)[0])
        elif (ct % 25 == 0) & (ct!=0):
            print('through {} days in {:.2f}'.format(ct,((time.time() -t)/60)))
        if d in starts:
            pickle.dump(l,open('nba/data/pickle/teamShotAllowed{}.pkl'.format(season),'wb'))   
            print('\tsaved {} season team Shots allowed'.format(season))
            l = []
            final = pd.DataFrame()
    except:
        missing.append(date)
print('Seasons uploaded, missing {:,} days of games'.format(len(missing)))
#pickle.dump(l,open('nba/data/pickle/shooting2.pkl','wb'))    

starting at 06:17
through 25 days in 2.65
through 50 days in 5.35
through 75 days in 7.70
through 100 days in 10.00
through 150 days in 87.18
	saved 2023-24 season team Shots allowed
through 175 days in 89.50
through 200 days in 92.07
through 225 days in 94.50
through 250 days in 96.86
through 275 days in 99.15
through 300 days in 101.73
	saved 2022-23 season team Shots allowed
through 325 days in 104.36
through 350 days in 106.73
through 375 days in 109.32
through 400 days in 111.79
through 425 days in 114.23
through 475 days in 118.96
	saved 2021-22 season team Shots allowed
through 500 days in 121.87
through 525 days in 124.44
through 550 days in 126.66
through 575 days in 129.33
through 600 days in 131.70
through 625 days in 134.00
through 650 days in 136.72
through 675 days in 139.00
through 700 days in 141.40
through 725 days in 144.05
through 750 days in 162.71
through 775 days in 165.17
	saved 2019-20 season team Shots allowed
through 800 days in 167.79
through 825 days in 170.

In [82]:
def clean_shot_columns(df):
    df.columns = ['{}_{}'.format(re.sub(' |-','_',a),b) if a!='' else b for a,b in df.columns]
    df = df.filter([col for col in df.columns if (re.search('ID$|^Restricted|^Mid|^In_The|^Right|^Left|^Above',col)!=None) &
                                (re.search('PCT$',col)==None)])
    return df

In [ ]:
class data:
    #probably want to get our db connection here and the last updated date
    def __init__(self):
        conn = sqlite3.connection('nba.db')
        lastGame = conn.cursor().exucute('select max(game_date) from games')
    
    def clean_lgdash_opp_columns(df,dribble):
        df = df.filter([col for col in df.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                        (df[col].dtype!=object)] )
        if dribble != False:
            df.columns = ['{}_{}'.format(col,dribble.replace(' ','_')) if col!='TEAM_ID' else col for col in df.columns] 
        return df
    
    def clean_shot_columns(df):
        df.columns = ['{}_{}'.format(re.sub(' |-','_',a),b) if a!='' else b for a,b in df.columns]
        df = df.filter([col for col in df.columns if (re.search('ID$|^Restricted|^Mid|^In_The|^Right|^Left|^Above',col)!=None) &
                                    (re.search('PCT$',col)==None)])
    return df
    
    
    def get_adv_box(self,gameids):
        l = []
        for gameid in gameids:
            box = BoxScoreAdvancedV3(game_id = gameid).get_data_frames()[0]
            d = box.filter([col for col in box.columns if (box[col].dtype!=object)])
            d['game_id'] = gameid
            l.append(d.to_dict(orient='records'))
        return l
    
    def get_first_buckets(self,gameid):
        for gameid in gameids:
            df = PlayByPlayV2(gameid).get_data_frames()[0]
            aind = df[(df.EVENTMSGTYPE==1) & (df.HOMEDESCRIPTION.notna())].PLAYER1_ID.idxmin()
            hind = df[(df.EVENTMSGTYPE==1) & (df.VISITORDESCRIPTION.notna())].PLAYER1_ID.idxmin()
            gd = {'gameid':gameid,'homeFirst':df.iloc[hind].PLAYER1_ID,'awayFirst':df.iloc[aind].PLAYER1_ID,
                  'first':df.iloc[min([aind,hind])].PLAYER1_ID}
        return gd
        
    def get_summary(self,gameid):
        l = []
        for gameid in gameids:
            b= BoxScoreSummaryV2(game_id=gameid).get_data_frames()
            df1 = b[5]
            df1 = df1.filter([col for col in df1.columns if (df1[col].dtype!=object) & (df1[col].sum()!=0) & (col!='GAME_SEQUENCE') ])
            df = df1.merge(b[1][['TEAM_ID','LARGEST_LEAD','PTS_FB','PTS_2ND_CHANCE','TOTAL_TURNOVERS']])
            df['GAME_ID'] = gameid
            l.append(df.to_dict(orient='records'))
        return l
        
    def get_opp_open_shot(self,dates):
        l = []
        for date in dates
            d = pd.to_datetime(date)
            season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
            oppShots = LeagueDashTeamShotLocations(measure_type_simple='Opponent',
                                              date_from_nullable = date,
                                               date_to_nullable = date,
                                           season=season
                               ).get_data_frames()[0]
            oppShots.columns = ['{}_{}'.format(re.sub(' |-','_',a),b) if a!='' else b for a,b in oppShots.columns]
            oppShots = oppShots.filter([col for col in oppShots.columns if re.search('_PCT$',col)==None])
            oppShots['GAME_DATE'] = date
            wide =  LeagueDashOppPtShot(date_from_nullable = date,
                              date_to_nullable = date, season=season,
                                  close_def_dist_range_nullable = '6+ Feet - Wide Open').get_data_frames()[0]
            wide = wide.filter([col for col in wide.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                        (wide[col].dtype!=object)] )
            wide.columns = [col if re.search('FG',col)==None else 'WIDEOPEN_{}'.format(col) for col in wide.columns]
            wide['GAME_DATE'] = date

            op = LeagueDashOppPtShot(date_from_nullable = date,
                              date_to_nullable = date,season=season,
                                  close_def_dist_range_nullable = '4-6 Feet - Open').get_data_frames()[0]
            op = op.filter([col for col in op.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                        (op[col].dtype!=object)] )
            op.columns = [col if re.search('FG',col)==None  else 'OPEN_{}'.format(col) for col in op.columns]
            op['GAME_DATE'] = date
            final = oppShots.merge(wide.merge(op,how='left',on=['TEAM_ID','GAME_DATE']),how='left',on=['TEAM_ID','GAME_DATE'])
            l.append(final.to_dict(orient='records'))
        return l
        
    def get_opp_dribble_shot(self,dates):
        l = []
        drib = ['0 Dribbles','1 Dribble','2 Dribbles','3-6 Dribbles','7+ Dribbles']
        d = pd.to_datetime(date)
        season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
        drb = pd.DataFrame()
        games = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
        games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
        for dribbleCount in drib:
            drbShots = LeagueDashOppPtShot(date_from_nullable = date,
                                date_to_nullable = date,
                                           season=season,
                                dribble_range_nullable=dribbleCount).get_data_frames()[0]
            df = lgDashOppColums(drbShots,dribbleCount)
            drb = pd.concat([drb,df])
            time.sleep(1)
        drb = drb.merge(games[games.GAME_DATE==d],how='left',on=['TEAM_ID'])
        drb = drb.groupby(['TEAM_ID','GAME_ID','GAME_DATE']).sum().reset_index()
        l.append(drb.to_dict(orient='records'))
        return l
    def get_player_shot_spots(self,dates):
        for date in dates:
            d = pd.to_datetime(date)
            season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
            df = LeagueDashPlayerShotLocations(date_from_nullable = date,
                                         date_to_nullable = date,
                                         season=season).get_data_frames()[0]
            df = shotcolumns(df)
            df['GAME_DATE'] = date
            temp = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
            temp = temp[temp.GAME_DATE==date]
            final = df.merge(temp,how='left',on=['GAME_DATE','TEAM_ID'])
        l.append(final.to_dict(orient='records'))
        return l
    
    def get_logs(season):
        log = PlayerGameLogs(season_nullable=season).get_data_frames()[0]
        roster = log.groupby(['SEASON_YEAR','PLAYER_ID','TEAM_ID']).GAME_DATE.agg(
                                                        firstGame = min,lastGame = max).reset_index()
        srchStr = 'SEASON_YEAR|MATCHUP|^TEAM_ABB|_PCT$|TEAM_NAME|^FG|REB$|_RANK$|WL|NICKNAME|_PTS$'
        df = log.filter([col for col in log.columns if re.search(srchStr,col)==None])
        return df
    
    def get_player_team(season):
        log = PlayerGameLogs(season_nullable=season).get_data_frames()[0]
        roster = log.groupby(['SEASON_YEAR','PLAYER_ID','TEAM_ID']).GAME_DATE.agg(
                                                        firstGame = min,lastGame = max).reset_index()
        return roster
    

In [ ]:
#advanced boxscore information for players
l = []
missing = []
t = time.time()
for ct,gameid in enumerate(game_ids):
    try:
        box = BoxScoreAdvancedV3(game_id = gameid).get_data_frames()[0]
        d = box.filter([col for col in box.columns if (box[col].dtype!=object)])
        d['game_id'] = gameid
        l.append(d.to_dict(orient='records'))
        if ct % 18 == 0:
            time.sleep(np.random.choice(range(1,12),1)[0])
        elif (ct % 100 == 0) & (ct!=0):
            print('through {} games in {:.2f}'.format(ct,((time.time() -t)/60)))
    except:
        missing.append(gameid)
print('Seasons uploaded, missing {:,} game ids'.format(len(missing)))
pickle.dump(l,open('nba/data/pickle/advbox.pkl','wb'))    

In [ ]:
l = pd.read_pickle('nba/data/pickle/advbox.pkl')

In [ ]:
adv = pd.DataFrame(l[1])

In [ ]:
lshots

In [ ]:
l_shots = pd.read_pickle('nba/data/pickle/plyrShots.pkl')

In [ ]:
lqtr = pd.read_pickle('nba/data/pickle/quarter2Info.pkl')

In [ ]:
lbox = pd.read_pickle('nba/data/pickle/box.pkl')

In [ ]:
lbox

In [ ]:
pd.DataFrame(lbox[1])

In [ ]:
l_s

In [ ]:
#from nba_api.stats.endpoints import PlayByPlayV2 - team play-by-play
l = []
missing = []
t = time.time()
for ct,gameid in enumerate(gids):
    try:
        b= BoxScoreSummaryV2(game_id=gameid).get_data_frames()
        df1 = b[5]
        df1 = df1.filter([col for col in df1.columns if (df1[col].dtype!=object) & (df1[col].sum()!=0) & (col!='GAME_SEQUENCE') ])
        df = df1.merge(b[1][['TEAM_ID','LARGEST_LEAD','PTS_FB','PTS_2ND_CHANCE','TOTAL_TURNOVERS']])
        df['GAME_ID'] = gameid
        l.append(df.to_dict(orient='records'))
        if ct % 18 == 0:
            time.sleep(np.random.choice(range(1,12),1)[0])
        elif (ct % 100 == 0) & (ct!=0):
            print('through {} games in {:.2f}'.format(ct,((time.time() -t)/60)))
    except:
        missing.append(gameid)
print('Seasons uploaded, missing {:,} game ids'.format(len(missing)))
pickle.dump(l,open('nba/data/pickle/quarter2Info.pkl','wb'))

In [ ]:
#from nba_api.stats.endpoints import PlayByPlayV2
l = []
missing = []
t = time.time()
for ct,gameid in enumerate(game_ids):
    try:
        df = PlayByPlayV2(gameid).get_data_frames()[0]
        aind = df[(df.EVENTMSGTYPE==1) & (df.HOMEDESCRIPTION.notna())].PLAYER1_ID.idxmin()
        hind = df[(df.EVENTMSGTYPE==1) & (df.VISITORDESCRIPTION.notna())].PLAYER1_ID.idxmin()
        gd = {'gameid':gameid,'homeFirst':df.iloc[hind].PLAYER1_ID,'awayFirst':df.iloc[aind].PLAYER1_ID,
              'first':df.iloc[min([aind,hind])].PLAYER1_ID}
        l.append(gd)
        if ct % 18 == 0:
            time.sleep(np.random.choice(range(1,12),1)[0])
        elif (ct % 100 == 0) & (ct!=0):
            print('through {} games in {:.2f}'.format(ct,((time.time() -t)/60)))
        del df
    except:
        missing.append(gameid)0
print('Seasons uploaded, missing {:,} game ids'.format(len(missing)))
pickle.dump(l,open('nba/data/pickle/baskets.pkl','wb'))

In [ ]:
df[(df.EVENTMSGTYPE==1) & (df.HOMEDESCRIPTION.notna())].HOMEDESCRIPTION.values[0]

In [ ]:
df[(df.EVENTMSGTYPE==1) & (df.VISITORDESCRIPTION.notna())].VISITORDESCRIPTION.values[0]

## general game information

In [ ]:
from nba_api.stats.endpoints import BoxScoreSummaryV2,CommonTeamRoster,PlayerGameLogs,PlayerAwards,TeamDetails
boxSum = BoxScoreSummaryV2(game_id='0022300790')
#adv = BoxScoreAdvancedV3(game_id='0022300790')
#trad = BoxScoreTraditionalV3(game_id='0022300790')
log = PlayerGameLogs(season_nullable='2023-24')
award = PlayerAwards(player_id='203076')
tinfo = CommonTeamRoster(team_id = '1610612756')

In [ ]:
def getLogs(season):
    log = PlayerGameLogs(season_nullable=season).get_data_frames()[0]
    srchStr = 'SEASON_YEAR|MATCHUP|^TEAM_ABB|_PCT$|TEAM_NAME|^FG|REB$|_RANK$|WL|NICKNAME|_PTS$'
    df = log.filter([col for col in log.columns if re.search(srchStr,col)==None])
    return df
    

In [ ]:
pshot = pd.read_pickle('nba/data/pickle/plyrShots.pkl')
fb = pd.read_pickle('nba/data/pickle/baskets.pkl')

In [ ]:
plyShot = pd.DataFrame(columns = ['PLAYER_ID', 'TEAM_ID', 'Restricted_Area_FGM', 'Restricted_Area_FGA', 'In_The_Paint_(Non_RA)_FGM', 'In_The_Paint_(Non_RA)_FGA', 'Mid_Range_FGM', 'Mid_Range_FGA', 'Left_Corner_3_FGM', 'Left_Corner_3_FGA', 'Right_Corner_3_FGM', 'Right_Corner_3_FGA', 'Above_the_Break_3_FGM', 'Above_the_Break_3_FGA', 'GAME_DATE', 'GAME_ID'])
for l in pshot:
    for d in l:
        if d['GAME_ID']=='0022300061':
            plyShot.loc[len(plyShot)] = list(d.values())

In [ ]:
#inactive player
boxSum.get_data_frames()[3]

In [ ]:
#officials
boxSum.get_data_frames()[2]

In [ ]:
boxSum.get_data_frames()[1][['TEAM_ID','LARGEST_LEAD','PTS_FB','PTS_2ND_CHANCE','TOTAL_TURNOVERS']]

In [ ]:
b= boxSum.get_data_frames

### Prototype view
- need player log data
    - this will have the layout shown in notes
    - for now will manually filter based on team names
    - need a rolling value as well will do 5 and 10
- create a function for the fair based on their pddvalue + 5% for a vig
- kelly criterion that will take in the actual odds

In [ ]:
with open('./nba/data/sql/createPlayerTable.sql','w') as file:
    f = '''
    CREATE TABLE box (
    playerId TEXT,
    teamId TEXT,
    gameId TEXT,
    min FLOAT,
    resAreaM INT,
    resAreaA INT,
    paintM INT,
    paintA INT,
    midM INT,
    midA INT,
    lcThreeM INT,
    lcThreeA INT,
    rcThreeM INT,
    rcThreeA INT,
    aboveThreeM INT,
    aboveThreeA INT,
    orb INT,
    orbContest INT,
    orbChances INT,
    orbDefer INT,
    avgOrbDist FLOAT,
    drb INT,
    drbContest INT,
    drbChances INT,
    drbDefer INT,
    avgDrbDist FLOAT,
    ftm INT,
    fta INT,
    ast INT,
    tov INT,
    stl INT,
    blk INT,
    shotBlk INT,
    pf INT,
    pfd INT,
    pts INT,
    plusMinus INT,
    dd INT,
    td INT,
    usage FLOAT,
    pace FLOAT,
    poss INT,
    offRating FLOAT,
    defRating FLOAT,
    teamFirstBasket INT,
    gameFirstBasket INT,
    PRIMARY KEY (playerId,gameId)
        FOREIGN KEY (playerId )
            REFERENCES players(playerId)
        FOREIGN KEY (gameId)
            REFERENCES games(gameId)
            
    
    )
    
    '''
    file.write(f)

In [ ]:
with open('./nba/data/sql/createGamesTable.sql','w') as file:
    f = '''
    CREATE TABLE games(
    gameId TEXT,
    gameDate TEXT,
    teamId TEXT,
    home INT,
    Q1 INT,
    Q2 INT,
    Q3 INT,
    Q4 INT,
    OT INT,
    inactive TEXT,
    pace FLOAT,
    offRating FLOAT,
    defRating FLOAT,
    oppOpenFG3A INT,
    oppOpenFG3M INT,
    oppOpenFG2A INT,
    oppOpenFG2M INT,
    oppWideOpenFG3A INT,
    oppWideOpenFG3M INT,
    oppWideOpenFG2A INT,
    oppWideOpenFG2M INT,
    oppResAreaA INT,
    oppResAreaM INT,
    oppPaintA INT,
    oppPaintM INT,
    oppMidA INT,
    oppMidM INT,
    oppLCA INT,
    oppLCM INT,
    oppRCA INT,
    oppRCM INT,
    oppAboveThreeA INT,
    oppAboveThreeM,
    drb0FG2M INT,
    drb0FG2A INT,
    drb0FG3M INT,
    drb0FG3A INT,
    drb1FG2M INT,
    drb1FG2A INT,
    drb1FG3M INT,
    drb1FG3A INT,
    drb2FG2M INT,
    drb2FG2A INT,
    drb2FG3M INT,
    drb2FG3A INT,
    drb36FG2M INT,
    drb36FG2A INT,
    drb36FG3M INT,
    drb36FG3A INT,
    drb7FG2M INT,
    drb7FG2A INT,
    drb7FG3M INT,
    drb7FG3A INT,
    PRIMARY KEY (gameId,teamId),
    FOREIGN KEY (teamId)
        REFERENCES teams(teamId)
    )
    '''
    file.write(f)

In [ ]:
with open('./nba/data/sql/createPlayerInfoTable.sql','w') as file:
    f='''
    CREATE TABLE players(
    playerId TEXT,
    playerName TEXT,
    playerHeight INT,
    playerWeight INT,
    playerWingspan INT,
    playerBirthday TEXT,
    playerAllNBA1 TEXT,
    playerAllNBA2 TEXT,
    playerAllNBA3 TEXT,
    playerAllDef1 TEXT,
    playerAllDef2 TEXT,
    playerAllstar TEXT,
    playerMVP TEXT,
    playerDPOY TEXT,
    playerChampionships TEXT,
    PRIMARY KEY (playerId)
    )
    '''
    file.write(f)

In [ ]:
with open('./nba/data/sql/createRosterTable.sql','w') as file:
    f = '''
    CREATE TABLE rosters(
    teamId TEXT,
    seasonYear TEXT,
    playerId TEXT,
    startDate TEXT,
    endDate TEXT,
    PRIMARY KEY (teamId,playerId,seasonYear),
    FOREIGN KEY (playerId) 
        REFERENCES players(playerId),
    FOREIGN KEY (teamId)
        REFERENCES teams(teamId)
    )
    
    '''
    file.write(f)

In [ ]:
with open('./nba/data/sql/createTeamInfoTable.sql','w') as file:
    f = '''
    CREATE TABLE teams(
    teamId TEXT,
    teamAbrv TEXT,
    teamName TEXT,
    teamConference TEXT,
    teamDivision TEXT,
    teamLong FLOAT,
    teamLAT FLOAT,
    PRIMARY KEY (teamId)
    
    )
    '''
    file.write(f)

In [ ]:
# cur.execute('''DROP TABLE rosters''')
# cur.execute('''DROP TABLE teams''')
cur.execute('''DROP TABLE box''')
# cur.execute('''DROP TABLE players''')
# cur.execute('''DROP TABLE games''')

In [ ]:
db = sqlite3.connect('./nba/data/database/nba.db')

In [ ]:
cur = db.cursor()

In [ ]:
# cur.execute(open('./nba/data/sql/createPlayerInfoTable.sql','r').read())
# cur.execute(open('./nba/data/sql/createRosterTable.sql','r').read())
# cur.execute(open('./nba/data/sql/createGamesTable.sql','r').read())
cur.execute(open('./nba/data/sql/createPlayerTable.sql','r').read())
# cur.execute(open('./nba/data/sql/createTeamInfoTable.sql','r').read())
db.commit()

In [ ]:
pd.read_pickle('./nba/data/pickle/')

In [ ]:
nba = data()

In [ ]:
ros = nba.get_roster(['2017-18','2018-19','2019-20','2020-21','2021-22','2022-23','2023-24'])

In [ ]:
pd.read_sql('''
select * from sqlite_master
where type = 'table'
and tbl_name = 'box'
''',db)

In [ ]:
pd.read_sql('PRAGMA table_info(box)',db)

In [ ]:
def insert_data(data,table):
    cols = pd.read_sql('''
    PRAGMA table_info({})
    '''.format(table),db).name.values
    rows = data.shape[0]
    v = '?' + ',?' * (len(cols)-1)
    cur.executemany('''insert into {t} values ({v})'''.format(t=table,v=v),data.values.tolist())
    return print('{} has been updated with {:,} rows'.format(table,rows))
        

In [ ]:
insert_data(ros,'rosters')

In [ ]:
tm = nba.get_team_info()

In [ ]:
insert_data(pd.DataFrame(tm),'teams')

In [ ]:
l = []
for gameid in gameids:
    df = PlayByPlayV2(gameid).get_data_frames()[0]
    aind = df[(df.EVENTMSGTYPE==1) & (df.HOMEDESCRIPTION.notna())].PLAYER1_ID.idxmin()
    hind = df[(df.EVENTMSGTYPE==1) & (df.VISITORDESCRIPTION.notna())].PLAYER1_ID.idxmin()
    gd = {'gameid':gameid,'homeFirst':1,'homePlayer':df.iloc[hind].PLAYER1_ID,'awayFirst':1,
          'awayPlayer':df.iloc[aind].PLAYER1_ID,'first':1,
          'firstPlayer':df.iloc[min([aind,hind])].PLAYER1_ID}
    l.append(gd)
df = pd.DataFrame(l)
home = df[df.homePlayer == df.firstPlayer][['gameid','homeFirst','homePlayer','first','firstPlayer']]
away = df[df.awayPlayer == df.firstPlayer][['gameid','awayFirst','awayPlayer','first','firstPlayer']]
del df

In [ ]:
h,a = nba.get_first_buckets(gameids)

In [ ]:
box = nba.get_advBox(gameids)

In [ ]:
bxdf = pd.DataFrame()

In [ ]:
bxcls = pd.read_sql('''PRAGMA table_info(box)''',db).name.values

In [ ]:
os.listdir('nba/data/pickle')

In [ ]:
bskts = pd.read_pickle('nba/data/pickle/baskets.pkl')

In [ ]:
def cleanBaskets(df):
    test['teamFirst'] = np.where(test.homeFirst!=test['first'],test.homeFirst,test.awayFirst)
    test['gameFirst'] = np.where(test.homeFirst==test['first'],test.homeFirst,test.awayFirst)
    game = test.filter(['gameid','gameFirst'])
    game['gameFirstBasket'] = 1
    team = test.filter(['gameid','teamFirst'])
    team['teamFirstBasket'] = 1
    df = pd.concat([game,team])
    df = df.rename(columns = {'gameid':'gameId'})
    df['playerId'] = df['gameFirst'].fillna(df.teamFirst)
    df[['gameId','playerId','gameFirstBasket','teamFirstBasket']]
    return df

In [ ]:
shot = pd.DataFrame([x for y in pd.read_pickle('nba/data/pickle/plyrShots.pkl')[:3] for x in y])

In [ ]:
def shot_clean():
    shot = pd.DataFrame([x for y in pd.read_pickle('nba/data/pickle/plyrShots.pkl') for x in y])
    return shot

In [ ]:
pd.DataFrame([y for x in box for y in x]).filter(['personId','game_id','usagePercentage','pace','possessions','offensiveRating','defensiveRating'])

In [ ]:
def 
def analytic_view(log,val,teams):
    '''
    Pass through the player log and the stat you are looking to evaluate to create a view to give basic information 
    on the player
    '''
    vals = [col for col in log.columns if col.find(val)>-1] + ['PLAYER_NAME','MIN']
    playing = log[log.TEAM_ABBREVIATION.isin(teams)]
    playing
    plval = playing[[val]]

In [ ]:
teams = ['CLE','DEN','LAL','BKN','PHI','TOR','MIA','WSH','OKC','HOU','DAL','NYK','CHI','MIN','GSW','SAS','UTA','SAC']

In [ ]:
playing = log[log.TEAM_ABBREVIATION.isin(teams)]

In [ ]:
df = playing[['PLAYER_NAME','GAME_DATE','MIN','FG3M']]

In [ ]:
df.groupby()

In [ ]:
drbsht = pd.read_pickle('./nba/data/pickle/drbshots.pkl')[0]

In [ ]:
drbsht[1]

In [ ]:
df.groupby(['PLAYER_NAME'])